In [ ]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
import litellm #for gemini
from openai import OpenAI #for OpenAI
import anthropic #for Claude 
import gradio as gr 

# set-up Frontier Model {Use one out of two method shown below}

In [ ]:
# --- Method 1: Paste your keys directly ---If a user does this and commits the file to GitHub, their secret key will be exposed to the public.
# Replace the placeholder text with your actual API keys.

# For OpenAI, your key usually starts with "sk-proj..."
os.environ['OPENAI_API_KEY'] = "PASTE_YOUR_OPENAI_API_KEY_HERE" 

# For Anthropic, your key usually starts with "sk-ant-api03..."
os.environ['ANTHROPIC_API_KEY'] = "PASTE_YOUR_ANTHROPIC_API_KEY_HERE"

# For Google Gemini
os.environ['GEMINI_API_KEY'] = "PASTE_YOUR_GEMINI_API_KEY_HERE"


# --- Method 2: Load keys from a .env file ( if you have api in .env) ---
# If you have your API keys in a .env file, you can use this method instead.
# First, uncomment the line below to install the required library:
# !pip install python-dotenv

# Then, uncomment the following lines of code:
# from dotenv import load_dotenv
# print("Loading keys from .env file...")
# load_dotenv()
# print("✅ Keys loaded.")


# --- Verification Step ---
# This part of the code checks if the keys have been loaded correctly.
print("Verifying API Keys...")

if os.getenv("OPENAI_API_KEY") and os.getenv("OPENAI_API_KEY") != "PASTE_YOUR_OPENAI_API_KEY_HERE":
    print("✅ OpenAI API Key is set.")
else:
    print("❌ OpenAI API Key is NOT set.")

if os.getenv("ANTHROPIC_API_KEY") and os.getenv("ANTHROPIC_API_KEY") != "PASTE_YOUR_ANTHROPIC_API_KEY_HERE":
    print("✅ Anthropic API Key is set.")
else:
    print("❌ Anthropic API Key is NOT set.")
    
if os.getenv("GEMINI_API_KEY") and os.getenv("GEMINI_API_KEY") != "PASTE_YOUR_GEMINI_API_KEY_HERE":
    print("✅ Gemini API Key is set.")
else:
    print("❌ Gemini API Key is NOT set.")

In [ ]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

litellm.completion()

# Start the Project

In [ ]:
# This cell defines a class to fetch and parse the text content from a webpage.

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."


# Function to stream from OpenAI models
def stream_gpt(prompt: str):
    openai_client = OpenAI()
    stream = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        stream=True,
    )
    
    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        yield response

# Function to stream from Anthropic models
def stream_claude(prompt: str):
    claude_client = anthropic.Anthropic()
    stream = claude_client.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=2048,
        messages=[
            {"role": "user", "content": prompt}
        ],
        system=system_message
    )
    
    response = ""
    with stream as s:
        for text in s.text_stream:
            response += text
            yield response

# Function to stream from Gemini models using LiteLLM
def stream_gemini(prompt: str):
    stream = litellm.completion(
        model="gemini/gemini-1.5-flash",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ],
        stream=True
    )

    response = ""
    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        response += content
        yield response

In [ ]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
# This cell uses Gradio to create and launch the web interface for the application.
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()